In [1]:
import boto3
import os
from dotenv import load_dotenv

from botocore.exceptions import ClientError
from constants import BUCKET_NAME, ACCOUNT_ID
from utility_functions import list_s3_bucket_objects

In [2]:
load_dotenv()

True

In [11]:
def get_aws_credentials(role):
    ROLE_ARN = f"arn:aws:iam::{ACCOUNT_ID}:role/{role}" 
    SESSION_NAME = f"{role}-notebook-session"
    session = boto3.Session(
        aws_access_key_id=os.getenv(f"AWS_{role}_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv(f"AWS_{role}_SECRET_ACCESS_KEY"),
    )
    sts_client = session.client("sts")
    try:
        assumed_role = sts_client.assume_role(
            RoleArn=ROLE_ARN,
            RoleSessionName=SESSION_NAME
        )
        creds = assumed_role["Credentials"]
        print(f"Assumed {role} role successfully.")
        return creds
    except ClientError as e:
        print("Error assuming HR role:", e)
        raise

def create_aws_client_for(creds, service_name):
    return boto3.client(
        service_name,
        aws_access_key_id=creds["AccessKeyId"],
        aws_secret_access_key=creds["SecretAccessKey"],
        aws_session_token=creds["SessionToken"],
        region_name="eu-north-1"
    )

In [12]:
engineer_s3_credentials = get_aws_credentials("ENGINEER")
hr_s3_credentials = get_aws_credentials("HR")
admin_s3_credentials = get_aws_credentials("ADMIN")

Assumed ENGINEER role successfully.
Assumed HR role successfully.
Assumed ADMIN role successfully.


In [13]:
engineer_s3_client = create_aws_client_for(engineer_s3_credentials, "s3")
hr_s3_client = create_aws_client_for(hr_s3_credentials, "s3")
admin_s3_client = create_aws_client_for(admin_s3_credentials, "s3")

In [14]:
list_s3_bucket_objects(engineer_s3_client)
list_s3_bucket_objects(hr_s3_client)
list_s3_bucket_objects(admin_s3_client)

No objects found in bucket 's3-rbac-in-data-lakes-experiments'.
No objects found in bucket 's3-rbac-in-data-lakes-experiments'.
No objects found in bucket 's3-rbac-in-data-lakes-experiments'.


In [15]:
engineer_kms_client = create_aws_client_for(engineer_s3_credentials, "kms")
hr_kms_client = create_aws_client_for(hr_s3_credentials, "kms")
admin_kms_client = create_aws_client_for(admin_s3_credentials, "kms")

In [16]:
hr_kms_client.list_keys()

{'Keys': [{'KeyId': '86c41c3f-fc21-4730-a20b-b755e5b63ebb',
   'KeyArn': 'arn:aws:kms:eu-north-1:501994300007:key/86c41c3f-fc21-4730-a20b-b755e5b63ebb'}],
 'Truncated': False,
 'ResponseMetadata': {'RequestId': '8a774f4c-e660-4cf6-b0a8-0c3f039ea21f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8a774f4c-e660-4cf6-b0a8-0c3f039ea21f',
   'cache-control': 'no-cache, no-store, must-revalidate, private',
   'expires': '0',
   'pragma': 'no-cache',
   'date': 'Fri, 05 Dec 2025 15:50:55 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '178',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [17]:
admin_kms_client.list_keys()

{'Keys': [{'KeyId': '5722129f-2136-4ef4-8b53-5a242b553f34',
   'KeyArn': 'arn:aws:kms:eu-north-1:501994300007:key/5722129f-2136-4ef4-8b53-5a242b553f34'},
  {'KeyId': '86c41c3f-fc21-4730-a20b-b755e5b63ebb',
   'KeyArn': 'arn:aws:kms:eu-north-1:501994300007:key/86c41c3f-fc21-4730-a20b-b755e5b63ebb'}],
 'Truncated': False,
 'ResponseMetadata': {'RequestId': '67f9a597-6ece-43e1-a78f-ad0a7c48b4b2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '67f9a597-6ece-43e1-a78f-ad0a7c48b4b2',
   'cache-control': 'no-cache, no-store, must-revalidate, private',
   'expires': '0',
   'pragma': 'no-cache',
   'date': 'Fri, 05 Dec 2025 15:54:14 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '315',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}